<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BMED%5D2033_Minimum_Operations_to_Make_a_Uni_Value_Grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2033. Minimum Operations to Make a Uni-Value Grid

https://leetcode.com/problems/minimum-operations-to-make-a-uni-value-grid/description/

You are given a 2D integer `grid` of size `m x n` and an integer `x`. In one operation, you can add `x` to or subtract `x` from any element in the `grid`.

A **uni-value** grid is a grid where **all the elements** of it are **equal**.

Return the minimum number of operations to make the `grid` **uni-value**. If it is not possible, return `-1`.

**Constraints:**
- `m == grid.length`
- `n == grid[i].length`
- `1 <= m, n <= 10`<sup>`5`</sup>
- `1 <= m * n <= 10`<sup>`5`</sup>
- `1 <= x, grid[i][j] <= 10`<sup>`4`</sup>

**Example 1:**

- Input: `grid = [[2,4],[6,8]], x = 2`

- Output: `4`

- Explanation:

    ![example1](https://drive.google.com/uc?id=12tITAM4wL-sLCjqSIi9yp7CMtRa3w4T_)

        We can make every element equal to 4 by doing the following:
        - Add x to 2 once.
        - Subtract x from 6 once.
        - Subtract x from 8 twice.
        A total of 4 operations were used.

**Example 2:**

- Input: `grid = [[1,5],[2,3]], x = 1`

- Output: `5`

- Explanation: We can make every element equal to 3.

    ![example2](https://drive.google.com/uc?id=19UJz08Ctj34xouIedmL5WvRlOrQGFNxc)


**Example 3:**

- Input: `grid = [[1,2],[3,4]], x = 2`

- Output: `-1`

- Explanation: It is impossible to make every element equal.

    ![example2](https://drive.google.com/uc?id=1EjwctEMTnxihjKXe8_Z8Ma4EUbyb0O_D)


In [ ]:
test_cases = [
    {
        "input": {
            "grid": [[2,4],[6,8]],
            "x": 2,
        },
        "output": 4
    },
    {
        "input": {
            "grid": [[1,5],[2,3]],
            "x": 1,
        },
        "output": 5
    },
    {
        "input": {
            "grid": [[1,2],[3,4]],
            "x": 2,
        },
        "output": -1
    },
    {
        "input": {
            "grid": [[146]],
            "x": 86,
        },
        "output": 0
    },
    {
        "input": {
            "grid": [[931,128],[639,712]],
            "x": 73,
        },
        "output": 12
    },
    {
        "input": {
            "grid": [[454,328,160,286,664],[496,538,748,244,286],[34,244,454,706,790],[496,538,832,958,328],[370,874,370,874,286]],
            "x": 42,
        },
        "output": 122
    },
    {
        "input": {
            "grid": [[596,904,960,232,120,932,176],[372,792,288,848,960,960,764],[652,92,904,120,680,904,120],[372,960,92,680,876,624,904],[176,652,64,344,316,764,316],[820,624,848,596,960,960,372],[708,120,456,92,484,932,540]],
            "x": 28,
        },
        "output": 473
    },
    {
        "input": {
            "grid": [[529,529,989],[989,529,345],[989,805,69]],
            "x": 92,
        },
        "output": 25
    },
]

In [ ]:
def run_test_cases(solution, function_name):
    for i in range(len(test_cases)):
        run_test_case(solution, function_name, i)

def run_test_case(solution, function_name, i):
    test_case = test_cases[i]
    result = getattr(solution, function_name)(**test_case["input"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

# Approach 1

### Observations

- If `x` is even, to make `grid` uni-value, all elements of `gird` must be even or odd, not both.

- If `uni_value` is the uni-value of `grid`, then `abs(grid[i][j] - uni_value) % x mod 0`

- The solution is converting 2D-array `grid` to sorted 1D-array `nums`. After that find the average value `mid = sum(nums)/len(nums)`.

- For all elements `nums[i]` smaller than `mid`,  `nums[i] mod x = s` must be the same

- For all elements `nums[i]` bigger than `mid`,  `nums[i] mod x = l` must be the same

- `(s + l) mod x` must be `0`

- Starting from initialized `mid`, find the nearest value that for all `i`, `abs(nums[i] - mid) mod x = 0`.

- Compute the operation cost to convert all element of `nums` to mid.

- Let `left`, `right` are left part and right part of `nums` which divided by `mid`

   - If `left.length > right.length`, adjust `mid` by subtract it by `x`

   - Else add `x` into `mid`

   - Compute new operation cost using new `mid`

   - If cost smaller than previous cost, continue adjusting `mid`, else return smallest cost.

### Analysis

- Time complexity:


### Implementation

In [ ]:
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
import math

class Solution:
    def minOperations(self, grid: List[List[int]], x: int) -> int:
        m, n = len(grid), len(grid[0])
        nums = [grid[i][j] for i in range(m) for j in range(n)]
        nums.sort()

        if len(nums) == 1:
            return 0

        mid = sum(nums) // len(nums)

        s = (mid - nums[0]) % x
        l = (nums[-1] - mid) % x

        if (s + l) % x != 0:
            return -1

        mid = mid - s
        min_ans = math.inf

        while True:
            count_ope_left, count_element_left = self.count_left(mid, nums, x)
            if count_ope_left == -1:
                return -1

            count_ope_right, count_element_right = self.count_rigt(mid, nums, x)
            if count_ope_right == -1:
                return -1

            ans = count_ope_left + count_ope_right
            if ans < min_ans:
                min_ans = ans
                if count_element_left >= count_element_right:
                    mid -= x
                else:
                    mid += x
            else:
                break
        return min_ans

    def count_left(self, mid, nums, x):
        i = 0
        operation_count = 0
        while True:
            if i == len(nums) or nums[i] > mid:
                break
            if (mid - nums[i]) % x != 0:
                return -1, None
            else:
                operation_count += (mid - nums[i]) // x
            i += 1
        return operation_count, i

    def count_rigt(self, mid, nums, x):
        j = len(nums) - 1
        operation_count = 0
        while True:
            if j == -1 or nums[j] <= mid:
                break
            if (nums[j] - mid) % x != 0:
                return -1, None
            else:
                operation_count += (nums[j] - mid) // x

            j -= 1

        return operation_count, len(nums) - 1 - j

In [ ]:
run_test_cases(Solution(), "minOperations")

In [ ]:
run_test_case(Solution(), "minOperations", 6)

# Approach 2: Leetcode's Editorial - Sorting and Median

![approach2](https://drive.google.com/uc?id=1wDczi5IfAFV3jqud-CXbnCWGSX6pMlsI)

In [ ]:
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
import math

class Solution:
    def minOperations(self, grid: List[List[int]], x: int) -> int:
        m, n = len(grid), len(grid[0])
        nums = [grid[i][j] for i in range(m) for j in range(n)]
        nums.sort()

        # checking if all element modulo with x have same remainer
        remainder = nums[0] % x
        for num in nums:
            if num % x != remainder:
                return -1

        # find mediand and operation cost to make all elements into median
        median = nums[len(nums)//2]
        cost = 0
        for num in nums:
            cost += abs(num - median) // x
        return cost

In [ ]:
run_test_cases(Solution(), "minOperations")